# Expliquer les Préférences : Pourquoi une cellule est-elle plus bénigne qu'une autre ?

**Question clé** : Comment expliquer à un patient ou à un médecin pourquoi le modèle considère qu'une cellule A est "plus bénigne" qu'une cellule B ?

Ce notebook propose une méthode pour **décomposer** cette comparaison en **arguments simples et compréhensibles** appelés *trade-offs* (compromis).

In [14]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from typing import List, Set, Optional, Dict
import warnings
warnings.filterwarnings('ignore')

## 1. Chargement des données

Le dataset Wisconsin Breast Cancer contient 683 échantillons de cellules avec 9 caractéristiques mesurées sur une échelle de 1 à 10 :

| Critère | Description | Interprétation |
|---------|-------------|----------------|
| ClumpThickness | Épaisseur des amas cellulaires | Plus c'est épais, plus c'est suspect |
| UniformityOfCellSize | Uniformité de la taille des cellules | Les cellules cancéreuses varient en taille |
| UniformityOfCellShape | Uniformité de la forme | Les cellules cancéreuses sont irrégulières |
| MarginalAdhesion | Adhésion marginale | Les cellules cancéreuses perdent leur adhérence |
| SingleEpithelialCellSize | Taille des cellules épithéliales | Cellules cancéreuses souvent agrandies |
| BareNuclei | Noyaux nus | Plus fréquents dans les tumeurs malignes |
| BlandChromatin | Chromatine fade | Texture anormale = signe de malignité |
| NormalNucleoli | Nucléoles normaux | Les nucléoles proéminents sont suspects |
| Mitoses | Activité mitotique | Division cellulaire rapide = danger |

In [15]:
# Charger les données
df = pd.read_csv('content/breastcancer_processed.csv')

noms_criteres = {
    'ClumpThickness': 'Épaisseur des amas',
    'UniformityOfCellSize': 'Uniformité taille',
    'UniformityOfCellShape': 'Uniformité forme',
    'MarginalAdhesion': 'Adhésion marginale',
    'SingleEpithelialCellSize': 'Taille épithéliale',
    'BareNuclei': 'Noyaux nus',
    'BlandChromatin': 'Chromatine',
    'NormalNucleoli': 'Nucléoles',
    'Mitoses': 'Mitoses'
}

feature_names = df.columns[1:].tolist()
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values  # 0 = Bénin, 1 = Malin

print(f" Dataset : {len(y)} cellules")
print(f"   - Bénignes : {np.sum(y == 0)} ({100*np.sum(y == 0)/len(y):.1f}%)")
print(f"   - Malignes : {np.sum(y == 1)} ({100*np.sum(y == 1)/len(y):.1f}%)")

 Dataset : 683 cellules
   - Bénignes : 444 (65.0%)
   - Malignes : 239 (35.0%)


In [16]:
df.head()

,Benign,ClumpThickness,UniformityOfCellSize,UniformityOfCellShape,MarginalAdhesion,SingleEpithelialCellSize,BareNuclei,BlandChromatin,NormalNucleoli,Mitoses
0,0,5,1,1,1,2,1,3,1,1
1,0,5,4,4,5,7,10,3,2,1
2,0,3,1,1,1,2,2,3,1,1
3,0,6,8,8,1,3,4,3,7,1
4,0,4,1,1,3,2,1,3,1,1


## 2. Le modèle : Régression logistique

On utilise une **régression logistique** qui calcule un **score de malignité** pour chaque cellule :

$$\text{Score}(\text{cellule}) = \sum_{i=1}^{9} w_i \times \text{valeur}_i$$

- **Score élevé** → la cellule ressemble à une cellule maligne
- **Score bas** → la cellule ressemble à une cellule bénigne

Les poids $w_i$ indiquent l'importance de chaque critère pour prédire la malignité.

In [17]:
# Entraîner le modèle
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X, y)

# Évaluation
scores_cv = cross_val_score(model, X, y, cv=5, scoring='accuracy')
print(f" Précision du modèle : {scores_cv.mean():.1%}")

# Poids du modèle
weights = model.coef_[0]

print("\n Importance des critères (poids du modèle) :")
print("-" * 50)
for name, w in sorted(zip(feature_names, weights), key=lambda x: -x[1]):
    bar = '█' * int(w * 10)
    print(f"  {noms_criteres[name]:<20} : {w:.3f} {bar}")

 Précision du modèle : 96.6%

 Importance des critères (poids du modèle) :
--------------------------------------------------
  Épaisseur des amas   : 0.525 █████
  Mitoses              : 0.483 ████
  Chromatine           : 0.433 ████
  Noyaux nus           : 0.381 ███
  Adhésion marginale   : 0.321 ███
  Uniformité forme     : 0.312 ███
  Nucléoles            : 0.211 ██
  Taille épithéliale   : 0.097 
  Uniformité taille    : 0.011 


## 3. Comparer deux cellules : le problème

Quand on compare deux cellules A et B, on veut expliquer **pourquoi A est plus bénigne que B** (ou inversement).

### Les trois types de critères

Pour chaque critère, on regarde la contribution au score :

- **Avantages (pros)** : critères où A est meilleure (plus bénigne) que B
- **Inconvénients (cons)** : critères où A est moins bien que B  
- **Neutres** : critères où A et B sont égales

### L'idée des trade-offs

Pour expliquer que A est globalement plus bénigne, on doit montrer comment ses **avantages compensent ses inconvénients**. C'est ce qu'on appelle un **trade-off** (compromis).

In [18]:
class ComparaisonCellules:
    """
    Compare deux cellules et identifie les avantages/inconvénients.

    Convention : on compare A vs B, où A est la cellule "plus bénigne".
    - Avantage : A a une valeur plus basse (meilleure) que B sur ce critère
    - Inconvénient : A a une valeur plus haute (pire) que B sur ce critère
    """

    def __init__(self, cellule_A: np.ndarray, cellule_B: np.ndarray,
                 weights: np.ndarray, feature_names: List[str]):
        self.A = cellule_A
        self.B = cellule_B
        self.weights = weights
        self.feature_names = feature_names

        # Scores de malignité
        self.score_A = np.dot(weights, cellule_A)
        self.score_B = np.dot(weights, cellule_B)

        # Contribution de chaque critère : omega_i = w_i * (B_i - A_i)
        # omega > 0 signifie que A est meilleure sur ce critère
        self.omega = weights * (cellule_B - cellule_A)

        # Classification des critères
        self.avantages = set(np.where(self.omega > 0)[0])   # A meilleure
        self.inconvenients = set(np.where(self.omega < 0)[0])  # A moins bien
        self.neutres = set(np.where(self.omega == 0)[0])

        # A est-elle vraiment plus bénigne ?
        self.difference = self.score_B - self.score_A  # > 0 si A plus bénigne
        self.A_plus_benigne = self.difference >= 0

    def afficher_comparaison(self):
        """Affiche un résumé visuel de la comparaison."""
        print("\n" + "=" * 70)
        print(f" COMPARAISON DE DEUX CELLULES")
        print("=" * 70)

        print(f"\n Scores de malignité :")
        print(f"   Cellule A : {self.score_A:.2f}")
        print(f"   Cellule B : {self.score_B:.2f}")
        print(f"   → {'A est plus bénigne' if self.A_plus_benigne else 'B est plus bénigne'} "
              f"(différence : {abs(self.difference):.2f})")

        print(f"\n Détail critère par critère :")
        print(f"{'Critère':<22} {'A':>5} {'B':>5} {'Diff':>8} {'Verdict':<15}")
        print("-" * 60)

        for i, name in enumerate(self.feature_names):
            diff = self.omega[i]
            if diff > 0:
                verdict = " Avantage A"
            elif diff < 0:
                verdict = " Inconvénient A"
            else:
                verdict = " Égalité"

            print(f"{noms_criteres[name]:<22} {self.A[i]:>5.0f} {self.B[i]:>5.0f} "
                  f"{diff:>+8.2f} {verdict}")

        print("\n Résumé :")
        print(f"    {len(self.avantages)} avantage(s) pour A")
        print(f"    {len(self.inconvenients)} inconvénient(s) pour A")
        print(f"    {len(self.neutres)} critère(s) neutre(s)")

## 4. Les types d'explications (trade-offs)

Pour expliquer pourquoi A est plus bénigne malgré certains inconvénients, on utilise trois types d'arguments :

###  Type 1 : Échange simple (1-1)
"L'avantage sur le critère X compense l'inconvénient sur le critère Y"

*Exemple : "La cellule A a des amas plus épais que B, mais sa chromatine est bien meilleure, ce qui compense."*

###  Type 2 : Argument fort (1-m)
"Un seul avantage majeur compense plusieurs petits inconvénients"

*Exemple : "Bien que A soit légèrement moins bien sur l'uniformité et l'adhésion, son excellente chromatine compense tout."*

###  Type 3 : Accumulation (m-1)
"Plusieurs petits avantages s'additionnent pour compenser un gros inconvénient"

*Exemple : "L'inconvénient majeur sur les mitoses est compensé par les avantages combinés sur la chromatine et les noyaux nus."*

In [19]:
class TradeOff:
    """
    Un trade-off est un compromis entre des avantages et des inconvénients.
    """

    def __init__(self, avantages: Set[int], inconvenients: Set[int],
                 comparaison: ComparaisonCellules):
        self.avantages = avantages
        self.inconvenients = inconvenients
        self.comparaison = comparaison

        # Force du trade-off = somme des contributions
        self.force = sum(comparaison.omega[i] for i in avantages | inconvenients)
        self.est_valide = self.force >= 0  # Les avantages compensent-ils ?

    def get_type(self) -> str:
        """Retourne le type de trade-off."""
        n_av, n_inc = len(self.avantages), len(self.inconvenients)
        if n_av == 1 and n_inc == 1:
            return "échange"
        elif n_av == 1:
            return "argument_fort"
        else:
            return "accumulation"

    def expliquer(self) -> str:
        """Génère une explication en français."""
        comp = self.comparaison
        av_noms = [noms_criteres[comp.feature_names[i]] for i in self.avantages]
        inc_noms = [noms_criteres[comp.feature_names[i]] for i in self.inconvenients]

        type_to = self.get_type()

        if type_to == "échange":
            av_idx = list(self.avantages)[0]
            inc_idx = list(self.inconvenients)[0]
            return (f" Échange : La meilleure {av_noms[0].lower()} de A "
                   f"({comp.A[av_idx]:.0f} vs {comp.B[av_idx]:.0f}) "
                   f"compense sa moins bonne {inc_noms[0].lower()} "
                   f"({comp.A[inc_idx]:.0f} vs {comp.B[inc_idx]:.0f})")

        elif type_to == "argument_fort":
            av_idx = list(self.avantages)[0]
            inc_str = ", ".join(inc_noms)
            return (f" Argument fort : L'excellent score de A sur {av_noms[0].lower()} "
                   f"({comp.A[av_idx]:.0f} vs {comp.B[av_idx]:.0f}) "
                   f"compense à lui seul les faiblesses sur : {inc_str}")

        else:  # accumulation
            inc_idx = list(self.inconvenients)[0]
            av_str = ", ".join(av_noms)
            return (f" Accumulation : Les avantages combinés de A sur {av_str} "
                   f"compensent ensemble le désavantage sur {inc_noms[0].lower()} "
                   f"({comp.A[inc_idx]:.0f} vs {comp.B[inc_idx]:.0f})")

In [20]:
class ExplicationComplete:
    """
    Une explication complète est un ensemble de trade-offs qui couvre
    tous les inconvénients de la cellule A.
    """

    def __init__(self, trade_offs: List[TradeOff]):
        self.trade_offs = trade_offs
        self.longueur = len(trade_offs)

    def est_valide(self) -> bool:
        return all(to.est_valide for to in self.trade_offs)

    def afficher(self):
        """Affiche l'explication complète."""
        print("\n" + "─" * 70)
        print(" EXPLICATION DE LA PRÉFÉRENCE")
        print("─" * 70)

        if self.longueur == 0:
            print("\n Dominance totale : A est meilleure que B sur TOUS les critères !")
            print("   → Aucun compromis nécessaire.")
            return

        print(f"\nPourquoi A est plus bénigne malgré certains inconvénients ?")
        print(f"Voici {self.longueur} argument(s) qui l'expliquent :\n")

        for i, to in enumerate(self.trade_offs, 1):
            print(f"{i}. {to.expliquer()}")
            print()

## 5. Algorithmes pour trouver les explications

In [21]:
def expliquer_par_echanges(comparaison: ComparaisonCellules) -> Optional[ExplicationComplete]:
    """
    Algorithme glouton pour expliquer avec des échanges simples (1 avantage ↔ 1 inconvénient).

    Principe : On associe chaque inconvénient à l'avantage le plus fort disponible,
    en traitant les critères par ordre de force décroissante.
    """
    if not comparaison.A_plus_benigne:
        return None

    # Cas de dominance : aucun inconvénient
    if len(comparaison.inconvenients) == 0:
        return ExplicationComplete([])

    # Trier tous les critères par force absolue
    criteres = list(comparaison.avantages | comparaison.inconvenients)
    criteres_tries = sorted(criteres, key=lambda i: abs(comparaison.omega[i]), reverse=True)

    avantages_disponibles = []
    trade_offs = []

    for idx in criteres_tries:
        if idx in comparaison.avantages:
            avantages_disponibles.append(idx)
        else:  # C'est un inconvénient
            if not avantages_disponibles:
                return None  # Pas d'avantage disponible pour compenser

            # Prendre l'avantage le plus fort
            meilleur_avantage = avantages_disponibles.pop(0)
            to = TradeOff({meilleur_avantage}, {idx}, comparaison)

            if not to.est_valide:
                return None  # L'avantage ne suffit pas à compenser

            trade_offs.append(to)

    return ExplicationComplete(trade_offs)


def expliquer_par_arguments_forts(comparaison: ComparaisonCellules) -> Optional[ExplicationComplete]:
    """
    Trouve une explication où chaque avantage peut compenser plusieurs inconvénients.

    Utilise une approche gloutonne : chaque avantage "mange" autant d'inconvénients
    qu'il peut en compenser.
    """
    if not comparaison.A_plus_benigne:
        return None

    if len(comparaison.inconvenients) == 0:
        return ExplicationComplete([])

    # Trier les avantages du plus fort au plus faible
    avantages_tries = sorted(comparaison.avantages,
                             key=lambda i: comparaison.omega[i], reverse=True)

    inconvenients_restants = set(comparaison.inconvenients)
    trade_offs = []

    for av in avantages_tries:
        if not inconvenients_restants:
            break

        force_av = comparaison.omega[av]
        # Trier les inconvénients du plus faible au plus fort (en valeur absolue)
        inc_tries = sorted(inconvenients_restants, key=lambda i: abs(comparaison.omega[i]))

        # Ajouter des inconvénients tant que l'avantage les compense
        inc_couverts = set()
        force_courante = force_av

        for inc in inc_tries:
            nouvelle_force = force_courante + comparaison.omega[inc]
            if nouvelle_force >= 0:
                inc_couverts.add(inc)
                force_courante = nouvelle_force

        if inc_couverts:
            trade_offs.append(TradeOff({av}, inc_couverts, comparaison))
            inconvenients_restants -= inc_couverts

    if inconvenients_restants:
        return None

    return ExplicationComplete(trade_offs)


def expliquer_par_accumulation(comparaison: ComparaisonCellules) -> Optional[ExplicationComplete]:
    """
    Trouve une explication où plusieurs avantages s'accumulent pour compenser
    chaque inconvénient.

    Traite les inconvénients du plus faible au plus fort.
    """
    if not comparaison.A_plus_benigne:
        return None

    if len(comparaison.inconvenients) == 0:
        return ExplicationComplete([])

    # Trier les inconvénients du plus faible au plus fort (en valeur absolue)
    inc_tries = sorted(comparaison.inconvenients,
                       key=lambda i: abs(comparaison.omega[i]))

    avantages_restants = set(comparaison.avantages)
    trade_offs = []

    for inc in inc_tries:
        if not avantages_restants:
            return None

        force_inc = comparaison.omega[inc]  # Négatif
        # Trier les avantages du plus fort au plus faible
        av_tries = sorted(avantages_restants,
                          key=lambda i: comparaison.omega[i], reverse=True)

        # Accumuler des avantages jusqu'à compenser l'inconvénient
        av_utilises = set()
        force_courante = force_inc

        for av in av_tries:
            av_utilises.add(av)
            force_courante += comparaison.omega[av]
            if force_courante >= 0:
                break

        if force_courante < 0:
            return None

        trade_offs.append(TradeOff(av_utilises, {inc}, comparaison))
        avantages_restants -= av_utilises

    return ExplicationComplete(trade_offs)


def trouver_meilleure_explication(comparaison: ComparaisonCellules) -> Optional[ExplicationComplete]:
    """
    Essaie toutes les stratégies et retourne la meilleure explication.
    """
    # Essayer dans l'ordre de préférence
    for methode in [expliquer_par_arguments_forts,
                    expliquer_par_echanges,
                    expliquer_par_accumulation]:
        explication = methode(comparaison)
        if explication is not None:
            return explication

    return None

## 6. Exemples concrets

Comparons des cellules réelles du dataset pour illustrer les explications.

In [22]:
def analyser_et_expliquer(idx_A: int, idx_B: int, label_A: str = "A", label_B: str = "B"):
    """
    Analyse complète de deux cellules avec explication.
    """
    comp = ComparaisonCellules(X[idx_A], X[idx_B], weights, feature_names)

    print(f"\n{'='*70}")
    print(f" Comparaison : Cellule {label_A} (#{idx_A}) vs Cellule {label_B} (#{idx_B})")
    print(f"   Diagnostic réel : {label_A}={'Bénigne' if y[idx_A]==0 else 'Maligne'}, "
          f"{label_B}={'Bénigne' if y[idx_B]==0 else 'Maligne'}")

    comp.afficher_comparaison()

    if comp.A_plus_benigne:
        explication = trouver_meilleure_explication(comp)
        if explication:
            explication.afficher()
        else:
            print("\n Impossible de trouver une explication simple.")
    else:
        print(f"\n Attention : {label_B} est en fait plus bénigne que {label_A} !")

    return comp

### Exemple 1 : Cellule bénigne vs cellule maligne

Comparons une cellule clairement bénigne avec une cellule maligne.

In [23]:
# Trouver une cellule très bénigne et une très maligne
scores_malignite = X @ weights
indices_tries = np.argsort(scores_malignite)

# La cellule la plus bénigne
idx_tres_benigne = indices_tries[0]
# Une cellule très maligne
idx_tres_maligne = indices_tries[-1]

print("Exemple 1 : Cas extrêmes")
comp1 = analyser_et_expliquer(idx_tres_benigne, idx_tres_maligne,
                               "Très bénigne", "Très maligne")

Exemple 1 : Cas extrêmes

 Comparaison : Cellule Très bénigne (#481) vs Cellule Très maligne (#597)
   Diagnostic réel : Très bénigne=Bénigne, Très maligne=Maligne

 COMPARAISON DE DEUX CELLULES

 Scores de malignité :
   Cellule A : 2.77
   Cellule B : 26.30
   → A est plus bénigne (différence : 23.53)

 Détail critère par critère :
Critère                    A     B     Diff Verdict        
------------------------------------------------------------
Épaisseur des amas         1     8    +3.68  Avantage A
Uniformité taille          1    10    +0.10  Avantage A
Uniformité forme           1    10    +2.81  Avantage A
Adhésion marginale         1    10    +2.89  Avantage A
Taille épithéliale         1     6    +0.49  Avantage A
Noyaux nus                 1    10    +3.43  Avantage A
Chromatine                 1    10    +3.90  Avantage A
Nucléoles                  1    10    +1.90  Avantage A
Mitoses                    1    10    +4.35  Avantage A

 Résumé :
    9 avantage(s) pour A
   

### Exemple 2 : Deux cellules bénignes mais différentes

Comparons deux cellules bénignes pour voir les nuances.

In [24]:
# Trouver deux cellules bénignes avec des profils différents
indices_benignes = np.where(y == 0)[0]
scores_benignes = scores_malignite[indices_benignes]
ordre_benignes = np.argsort(scores_benignes)

# Une cellule très clairement bénigne
idx_benigne_claire = indices_benignes[ordre_benignes[0]]
# Une cellule bénigne mais avec un profil plus ambigu
idx_benigne_ambigue = indices_benignes[ordre_benignes[-1]]

print("\n Exemple 2 : Deux cellules bénignes, profils différents")
comp2 = analyser_et_expliquer(idx_benigne_claire, idx_benigne_ambigue,
                               "Bénigne claire", "Bénigne ambiguë")


 Exemple 2 : Deux cellules bénignes, profils différents

 Comparaison : Cellule Bénigne claire (#481) vs Cellule Bénigne ambiguë (#190)
   Diagnostic réel : Bénigne claire=Bénigne, Bénigne ambiguë=Bénigne

 COMPARAISON DE DEUX CELLULES

 Scores de malignité :
   Cellule A : 2.77
   Cellule B : 15.84
   → A est plus bénigne (différence : 13.07)

 Détail critère par critère :
Critère                    A     B     Diff Verdict        
------------------------------------------------------------
Épaisseur des amas         1     8    +3.68  Avantage A
Uniformité taille          1     4    +0.03  Avantage A
Uniformité forme           1     4    +0.94  Avantage A
Adhésion marginale         1     5    +1.28  Avantage A
Taille épithéliale         1     4    +0.29  Avantage A
Noyaux nus                 1     7    +2.28  Avantage A
Chromatine                 1     7    +2.60  Avantage A
Nucléoles                  1     8    +1.48  Avantage A
Mitoses                    1     2    +0.48  Avantage

### Exemple 3 : Cas intéressant avec trade-offs

Cherchons un cas où la cellule bénigne a vraiment des inconvénients à compenser.

In [25]:
# Chercher un cas avec au moins 2 inconvénients pour la cellule bénigne
np.random.seed(42)
cas_interessant_trouve = False

for _ in range(100):
    i = np.random.choice(indices_benignes)
    j = np.random.choice(np.where(y == 1)[0])

    comp_test = ComparaisonCellules(X[i], X[j], weights, feature_names)

    # On veut un cas où la cellule bénigne a au moins 2 inconvénients
    if comp_test.A_plus_benigne and len(comp_test.inconvenients) >= 2:
        print("\n Exemple 3 : Cas avec plusieurs compromis")
        comp3 = analyser_et_expliquer(i, j, "Bénigne", "Maligne")
        cas_interessant_trouve = True
        break

if not cas_interessant_trouve:
    print("Pas de cas avec plusieurs compromis trouvé dans l'échantillon.")


 Exemple 3 : Cas avec plusieurs compromis

 Comparaison : Cellule Bénigne (#285) vs Cellule Maligne (#279)
   Diagnostic réel : Bénigne=Bénigne, Maligne=Maligne

 COMPARAISON DE DEUX CELLULES

 Scores de malignité :
   Cellule A : 10.86
   Cellule B : 11.47
   → A est plus bénigne (différence : 0.62)

 Détail critère par critère :
Critère                    A     B     Diff Verdict        
------------------------------------------------------------
Épaisseur des amas         5     6    +0.53  Avantage A
Uniformité taille          3     1    -0.02  Inconvénient A
Uniformité forme           4     3    -0.31  Inconvénient A
Adhésion marginale         3     1    -0.64  Inconvénient A
Taille épithéliale         4     4    +0.00  Égalité
Noyaux nus                 5     5    +0.00  Égalité
Chromatine                 4     5    +0.43  Avantage A
Nucléoles                  7    10    +0.63  Avantage A
Mitoses                    1     1    +0.00  Égalité

 Résumé :
    3 avantage(s) pour A
  

## 7. Statistiques d'explicabilité

Quelle proportion des comparaisons peut-on expliquer avec chaque type de trade-off ?

In [26]:
def calculer_statistiques(n_echantillons: int = 2000):
    """
    Calcule les statistiques d'explicabilité sur un échantillon de comparaisons.
    """
    np.random.seed(42)
    n = len(X)

    # Générer des paires aléatoires où A est plus bénigne que B
    paires = []
    tentatives = 0
    while len(paires) < n_echantillons and tentatives < n_echantillons * 10:
        i, j = np.random.choice(n, 2, replace=False)
        if scores_malignite[i] < scores_malignite[j]:
            paires.append((i, j))
        tentatives += 1

    stats = {
        'total': len(paires),
        'dominance': 0,
        'echanges': 0,
        'arguments_forts': 0,
        'accumulation': 0,
        'explicable': 0
    }

    for i, j in paires:
        comp = ComparaisonCellules(X[i], X[j], weights, feature_names)

        if len(comp.inconvenients) == 0:
            stats['dominance'] += 1
            stats['explicable'] += 1
            continue

        if expliquer_par_echanges(comp) is not None:
            stats['echanges'] += 1

        if expliquer_par_arguments_forts(comp) is not None:
            stats['arguments_forts'] += 1

        if expliquer_par_accumulation(comp) is not None:
            stats['accumulation'] += 1

        if trouver_meilleure_explication(comp) is not None:
            stats['explicable'] += 1

    return stats

print(" Calcul des statistiques d'explicabilité...")
stats = calculer_statistiques(2000)

print(f"\n{'='*60}")
print("STATISTIQUES D'EXPLICABILITÉ")
print(f"{'='*60}")
print(f"\nSur {stats['total']} comparaisons analysées :")
print(f"\n{'Type d\'explication':<30} {'Nombre':>10} {'%':>10}")
print("-" * 52)
print(f"{' Dominance (aucun compromis)':<30} {stats['dominance']:>10} {100*stats['dominance']/stats['total']:>9.1f}%")
print(f"{' Échanges simples':<30} {stats['echanges']:>10} {100*stats['echanges']/stats['total']:>9.1f}%")
print(f"{' Arguments forts':<30} {stats['arguments_forts']:>10} {100*stats['arguments_forts']/stats['total']:>9.1f}%")
print(f"{' Accumulation':<30} {stats['accumulation']:>10} {100*stats['accumulation']/stats['total']:>9.1f}%")
print("-" * 52)
print(f"{' TOTAL EXPLICABLE':<30} {stats['explicable']:>10} {100*stats['explicable']/stats['total']:>9.1f}%")

 Calcul des statistiques d'explicabilité...

STATISTIQUES D'EXPLICABILITÉ

Sur 2000 comparaisons analysées :

Type d'explication                 Nombre          %
----------------------------------------------------
 Dominance (aucun compromis)         1166      58.3%
 Échanges simples                     637      31.9%
 Arguments forts                      702      35.1%
 Accumulation                         653      32.6%
----------------------------------------------------
 TOTAL EXPLICABLE                    1937      96.8%
